In [ ]:
%pip install -r requirements.txt

In [1]:
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

## load GloVe embeddings

In [2]:
embeddings_dict = {}
with open("glove.twitter.27B.100d.txt", 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        if len(vector)==100:
            embeddings_dict[word] = vector

In [3]:
def find_closest_embeddings(embedding):
    return sorted(embeddings_dict.keys(), key=lambda word: spatial.distance.euclidean(embeddings_dict[word], embedding))

In [4]:
find_closest_embeddings(embeddings_dict["people"])[:5]

['people', 'other', 'many', 'those', 'ppl']

## tetattack

In [3]:
import tensorflow as tf
import keras
import numpy as np
from keras.utils import to_categorical
from textattack.models.wrappers import ModelWrapper
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import PWWSRen2019

import numpy as np
from keras.utils import to_categorical
from keras import models
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import LSTM,Conv1D
from keras.layers import Bidirectional
from keras_preprocessing.sequence import pad_sequences

from nltk.tokenize import word_tokenize, RegexpTokenizer

/opt/mambaforge/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# load twitter data
import pandas as pd
data = pd.read_csv('train_plain.csv')
texts = data['raw_tweet'].values

In [67]:
from fast_ml.model_development import train_valid_test_split

df = data[['label','raw_tweet']]
X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(df, target = 'label', 
                                                                            train_size=0.8, valid_size=0.1, test_size=0.1, random_state=11)

In [68]:
from sklearn.preprocessing import OneHotEncoder,LabelEncoder

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_valid = to_categorical(y_valid)

In [101]:
vocab_size

14897

In [95]:
y_train = np.delete(y_train,1262,0)

In [69]:
from keras.preprocessing.text import Tokenizer
from keras.initializers import Constant
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [70]:
vocab_size = len(tokenizer.word_index.items())
embedding_matrix = np.zeros((vocab_size,100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [71]:
X_train_indices = tokenizer.texts_to_sequences(list(X_train['raw_tweet']))

X_train_indices = pad_sequences(X_train_indices, maxlen=100, padding='post')

X_test_indices = tokenizer.texts_to_sequences(list(X_test['raw_tweet']))

X_test_indices = pad_sequences(X_test_indices, maxlen=100, padding='post')

In [89]:
X_train_indices =np.delete(X_train_indices,1262,0)

In [90]:
def create_embedding_model(vocab_size, max_length):
    model = models.Sequential()
    model.add(layers.Embedding(vocab_size, 100, input_length=max_length,embeddings_initializer=Constant(embedding_matrix), trainable=False))
    model.add(Bidirectional(LSTM(128, activation='relu',return_sequences=True)))
    model.add(layers.Conv1D(10, 3, activation="relu"))
    model.add(layers.Conv1D(10, 3, activation="relu"))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Flatten())
    model.add(Dropout(0.5))
    #model.add(layers.Dense(10, activation="relu"))
    model.add(layers.Dense(2,  activation="sigmoid"))   
    return model

In [109]:
embedding_model = create_embedding_model(vocab_size=vocab_size, max_length=100)
embedding_model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 100, 100)          1489700   
                                                                 
 bidirectional_9 (Bidirectio  (None, 100, 256)         234496    
 nal)                                                            
                                                                 
 conv1d_18 (Conv1D)          (None, 98, 10)            7690      
                                                                 
 conv1d_19 (Conv1D)          (None, 96, 10)            310       
                                                                 
 max_pooling1d_9 (MaxPooling  (None, 48, 10)           0         
 1D)                                                             
                                                                 
 flatten_9 (Flatten)         (None, 480)              

In [110]:
opt = keras.optimizers.Adam(learning_rate=0.001)
embedding_model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])


In [111]:
from keras.callbacks import EarlyStopping

earlyStopping = EarlyStopping(monitor="val_loss", patience=3)

modelHistory = embedding_model.fit(X_train_indices, 
                                   y_train, 
                                   validation_data=(X_test_indices, y_test),
                                   epochs=32,
                                   batch_size = 32,
                                   callbacks=[earlyStopping],
                                   shuffle=False
                                   )

Epoch 1/32


136/136 [==============================] - 21s 138ms/step - loss: 161.5800 - accuracy: 0.6612 - val_loss: 0.5333 - val_accuracy: 0.7006
Epoch 2/32
136/136 [==============================] - 18s 135ms/step - loss: 0.5156 - accuracy: 0.7213 - val_loss: 0.4797 - val_accuracy: 0.8096
Epoch 3/32
136/136 [==============================] - 18s 133ms/step - loss: 0.4662 - accuracy: 0.7996 - val_loss: 0.4442 - val_accuracy: 0.8244
Epoch 4/32
136/136 [==============================] - 18s 133ms/step - loss: 0.4342 - accuracy: 0.8343 - val_loss: 0.4246 - val_accuracy: 0.8503
Epoch 5/32
136/136 [==============================] - 18s 134ms/step - loss: 0.7038 - accuracy: 0.8237 - val_loss: 0.4275 - val_accuracy: 0.8299
Epoch 6/32
136/136 [==============================] - 18s 136ms/step - loss: 0.3921 - accuracy: 0.8625 - val_loss: 0.4229 - val_accuracy: 0.8614
Epoch 7/32
136/136 [==============================] - 18s 135ms/step - loss: 0.3714 - accuracy: 0.8676 - val_loss: 0.4352 - val_accuracy: 0

In [112]:
embedding_model.save("embedding_model.h5")

## textattack
- generate augmentated samples
- test generated data on trained model

### generate augmented samples

In [113]:
vocabulary = tokenizer.word_index.items()

In [114]:
class CustomKerasModelWrapper(ModelWrapper):
    def __init__(self, model):
        self.model = model

    def __call__(self, text_input_list):

      x_transform = []
      
      tokens = tokenizer.texts_to_sequences(text_input_list)

      x_transform = pad_sequences(tokens, maxlen=100, padding='post')

      #print(x_transform)
       
      prediction = self.model.predict(x_transform)
      
      return prediction


CustomKerasModelWrapper(embedding_model)(["bad bad bad bad bad", "good good good good"])

1/1 [==============================] - 0s 289ms/step


array([[0.676403  , 0.3263657 ],
       [0.67079556, 0.33218792]], dtype=float32)

In [115]:
import textattack
dataset = textattack.datasets.Dataset(data[['raw_tweet','label']].values.tolist())


In [117]:
from textattack import AttackArgs
from textattack.datasets import Dataset
from textattack import Attacker

model_wrapper = CustomKerasModelWrapper(embedding_model)
#dataset = HuggingFaceDataset("rotten_tomatoes", None, "test", shuffle=True)

attack = PWWSRen2019.build(model_wrapper)

attack_args = AttackArgs(num_successful_examples=2000, log_to_csv="results1.csv", csv_coloring_style="html")

attacker = Attacker(attack, dataset, attack_args)

attacker.attack_dataset()

[nltk_data] Downloading package omw-1.4 to /home/kzeng4/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
textattack: Unknown if model of class <class 'keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path results1.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



1/1 [==============================] - 0s 41ms/step
--------------------------------------------- Result 1 ---------------------------------------------
[[1 (68%)]] --> [[[FAILED]]]

My step-dad had Alzheimer’s for 14 year. Biden talks and acts just like him. They are very similar in character and he got very mean and short tempered when he recognized his inability to speak clearly, say the words he was trying to remember and when he was embarrassed.




1/1 [==============================] - 0s 31ms/step
--------------------------------------------- Result 2 ---------------------------------------------
[[1 (54%)]] --> [[[SKIPPED]]]

What others are saying about the course: ""I'm very glad I found this program. It has made my attitude toward care giving more positive and hopeful. It has created a more pleasant environment for me and my husband."" thedawnmethod.com/training/ https://t.co/Jnd2gSm5JS




1/1 [==============================] - 0s 36ms/step


...

--------------------------------------------- Result 98 ---------------------------------------------
[[1 (72%)]] --> [[[SKIPPED]]]

My cousin’s mother died last fall from dementia and COVID complications and I don’t think he’s going to apply because “it’s the government” and “she didn’t really have COVID.” 🙄


1/1 [==============================] - 0s 30ms/step



[Succeeded / Failed / Skipped / Total] 52 / 26 / 21 / 99:   3%|▎         | 52/2000 [03:25<2:08:35,  3.96s/it]

--------------------------------------------- Result 99 ---------------------------------------------
[[0 (54%)]] --> [[[SKIPPED]]]

It’s the one thing that makes me mad about my dad’s Alzheimer’s. They put him on Aricept early on, but mom took him off it because “he wasn’t getting better”. Grrrrrr!




1/1 [==============================] - 0s 34ms/step


--------------------------------------------- Result 100 ---------------------------------------------
[[1 (73%)]] --> [[0 (51%)]]

_McClure It’s so [[important]] - my [[daughters]] [[school]] _Anthonys3 [[delivered]] the [[course]]. My [[mum]] had [[dementia]] and [[sadly]] [[died]] [[last]] [[year]]. [[Very]] [[committed]] to [[support]] education of dementia ❤️

_McClure It’s so [[significant]] - my [[girl]] [[educate]] _Anthonys3 [[pitch]] the [[trend]]. My [[silent]] had [[dementedness]] and [[deplorably]] [[fail]] [[finish]] [[class]]. [[really]] [[practice]] to [[confirm]] education of dementia ❤️


1/1 [==============================] - 0s 36ms/step


In [9]:
import re
test_text = '<font color = green>I</font> <font color = green>took</font> my'
re.sub('<.*?>','',test_text)

'I took my'

In [31]:
import pandas as pd
df_per = pd.read_csv('results.csv')
df_per[:5].style

,original_text,perturbed_text,original_score,perturbed_score,original_output,perturbed_output,ground_truth_output,num_queries,result_type
0,"My step-dad had Alzheimer’s for 14 year. Biden talks and acts just like him. They are very similar in character and he got very mean and short tempered when he recognized his inability to speak clearly, say the words he was trying to remember and when he was embarrassed.","My step-dad had Alzheimer’s for 14 twelvemonth. Biden talks and move just alike him. They are very alike in character and he arrest very mean and dead moderate when he recognized his inability to speak clearly, say the words he was trying to think and when he was embarrassed.",0.323358,0.531440,1,0,1,433,Successful
1,"What others are saying about the course: """"I'm very glad I found this program. It has made my attitude toward care giving more positive and hopeful. It has created a more pleasant environment for me and my husband."""" thedawnmethod.com/training/ https://t.co/Jnd2gSm5JS","What others are saying about the course: """"I'm very glad I found this program. It has made my attitude toward care giving more positive and hopeful. It has created a more pleasant environment for me and my husband."""" thedawnmethod.com/training/ https://t.co/Jnd2gSm5JS",0.617823,0.617823,1,1,0,1,Skipped
2,_Grant Dementia and senility is the equivalent of returning to a child like state so his comment is in keeping with his acuity.,_Grant Dementia and senility is the equivalent of regress to a child wish express so his notice is in preserve with his acuteness.,0.286069,0.573807,0,1,0,181,Successful
3,"I live in the US where there are SO many Spanish speakers. I initially chose Japanese, but quickly changed course because: 1. Spanish makes most sense career wise 2. My Main goal was to just be bilingual (I don't want alzheimer's) 3. I want my daughter to speak Spanish","I live in the US where there are SO many Spanish speakers. I initially chose Japanese, but quickly changed course because: 1. Spanish makes most sense career wise 2. My Main goal was to just be bilingual (I don't want alzheimer's) 3. I want my daughter to speak Spanish",0.646258,0.646258,1,1,0,1,Skipped
4,"Please let's be serious the poor man has inherited his father's gen ""Alzheimer's"" He is next walking in his underwear and talking to the man in the mirror","Please let's be severe the inadequate humanity has inherited his father's gen ""Alzheimers"" He is next walking in his underwear and sing to the world in the mirror",0.279635,0.547439,0,1,0,147,Successful


### test generated data on trained model

In [27]:
import re
text_no_color = [re.sub('<.*?>','',text) for text in df_per['perturbed_text']]
df_per['text_no_color'] = text_no_color

In [28]:
load_model = models.load_model('embedding_model.h5')

In [28]:
perturbed_indices = tokenizer.texts_to_sequences(df_per['text_no_color'])
perturbed_indices = pad_sequences(perturbed_indices, maxlen=100, padding='post')
y_perturbed = to_categorical(df_per['ground_truth_output'])
load_model.evaluate(perturbed_indices,y_perturbed)

NameError: name 'load_model' is not defined

In [35]:
print(load_model.metrics_names)

['loss', 'accuracy']


### retrained the model with generated augmented data
- which labeled should be used here?

In [21]:
df_per = pd.read_csv('results.csv',index_col=0)

In [22]:
import random
index = np.arange(0,len(df_per[df_per['result_type']!='Skipped'])-1,1,dtype = int)
random.Random(44).shuffle(index)
train_index = index[:2000]
test_index = index[2000:]

In [29]:
perturbed_train_indices = perturbed_indices[train_index]
perturbed_test_indices = perturbed_indices[test_index]
y_train_perturbed = y_perturbed[train_index]
y_test_perturbed = y_perturbed[test_index]

In [40]:
from keras.callbacks import EarlyStopping

earlyStopping = EarlyStopping(monitor="val_loss", patience=5)

modelHistory = embedding_model.fit(perturbed_train_indices, 
                                   y_train_perturbed, 
                                   validation_data=(perturbed_test_indices, y_test_perturbed),
                                   epochs=16,
                                   batch_size = 32,
                                   callbacks=[earlyStopping]
                                   )

Epoch 1/16
63/63 [==============================] - 11s 150ms/step - loss: 5102.0039 - accuracy: 0.6615 - val_loss: 0.7464 - val_accuracy: 0.7019
Epoch 2/16
63/63 [==============================] - 9s 136ms/step - loss: 13.4659 - accuracy: 0.6870 - val_loss: 0.5313 - val_accuracy: 0.7040
Epoch 3/16
63/63 [==============================] - 9s 146ms/step - loss: 0.6070 - accuracy: 0.7375 - val_loss: 0.5391 - val_accuracy: 0.8055
Epoch 4/16
63/63 [==============================] - 9s 144ms/step - loss: 0.5006 - accuracy: 0.7675 - val_loss: 0.4346 - val_accuracy: 0.7992
Epoch 5/16
63/63 [==============================] - 9s 140ms/step - loss: 0.4986 - accuracy: 0.8150 - val_loss: 0.4005 - val_accuracy: 0.8224
Epoch 6/16
63/63 [==============================] - 9s 143ms/step - loss: 0.4002 - accuracy: 0.8300 - val_loss: 0.3577 - val_accuracy: 0.8161
Epoch 7/16
63/63 [==============================] - 8s 131ms/step - loss: 0.3603 - accuracy: 0.8505 - val_loss: 0.3411 - val_accuracy: 0.8309
E

In [79]:
embedding_model.save("embedding_model_perturbed.h5")

### test manually labeled small validation set

In [13]:
pos_s = pd.read_csv('pos_s.csv',index_col = 0)

In [14]:
pos_s[:3]

,target,body
1744171,0,@seeks_the_dark My mother's favorite.
63452,0,'Szerelem' sung by Marta Szebesztyen on radio ...
2266207,0,My mom would have turned 48 today. When I was ...


In [17]:
load_model1 = models.load_model('embedding_model.h5')
load_model2 = models.load_model('embedding_model_perturbed.h5')

In [16]:
pos_s_indices = tokenizer.texts_to_sequences(pos_s['body'])
pos_s_indices = pad_sequences(pos_s_indices, maxlen=100, padding='post')
y_pos_s = to_categorical(pos_s['target'])
load_model1.evaluate(pos_s_indices,y_pos_s)

7/7 [==============================] - 1s 55ms/step - loss: 1.8495 - accuracy: 0.2850


[1.8495477437973022, 0.2849999964237213]

In [18]:
load_model2.evaluate(pos_s_indices,y_pos_s)

7/7 [==============================] - 1s 53ms/step - loss: 2.3872 - accuracy: 0.5750


[2.3871572017669678, 0.574999988079071]

In [29]:

prob1 = list(load_model1.predict(pos_s_indices)[:,0])
prob2 = list(load_model2.predict(pos_s_indices)[:,0])
result = pd.DataFrame({'prob1':prob1,'prob2':prob2,'label':pos_s['target']})

7/7 [==============================] - 0s 51ms/step


In [30]:
result

,prob1,prob2,label
1744171,0.674284,0.009138,0
63452,0.172710,0.390299,0
2266207,0.000609,0.987030,0
2235640,0.022364,0.992832,0
3381848,0.428398,0.602039,0
...,...,...,...
754334,0.068383,0.806038,0
2910493,0.743262,0.934948,1
2963130,0.120413,0.931122,0
2910422,0.566922,0.908382,1


In [27]:
# remove #,@
pos_s_plain = pd.read_csv('pos_s_plain.csv',index_col = 0)

In [20]:
plain_indices = tokenizer.texts_to_sequences(pos_s_plain['plain_tweet'])
plain_indices = pad_sequences(plain_indices, maxlen=100, padding='post')
y_plain = to_categorical(pos_s_plain['target'])

In [21]:
load_model1.evaluate(plain_indices,y_pos_s)
load_model2.evaluate(plain_indices,y_pos_s)

7/7 [==============================] - 0s 52ms/step - loss: 2.3672 - accuracy: 0.4400


[2.3671648502349854, 0.4399999976158142]